In [22]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


from sklearn.svm import SVC
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

In [25]:
df=pd.read_excel('proj2dataset.xlsx',header=None)
df

,0,1,2
0,1.160947,3.316569,1
1,0.384876,2.260154,1
2,0.761007,4.373630,1
3,1.614981,4.262774,1
4,0.098600,3.659726,1
...,...,...,...
95,2.358362,0.411036,-1
96,3.844820,1.789896,-1
97,3.935609,0.906999,-1
98,5.286971,3.140954,-1


In [20]:
data=df.values
